In [4]:
import pandas as pd

df = pd.read_csv("data/train-part-1-processed.csv")



In [5]:
import numpy as np
from ast import literal_eval

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [16]:
df['embedding'].apply(type).unique()

array([<class 'numpy.ndarray'>], dtype=object)

In [17]:
features = np.hstack([
    np.vstack(df.embedding.values),
    np.vstack(df.embedding_b.values)
])
X_train, X_test, y_train, y_test = train_test_split(features, df.weight_a, test_size=0.2, random_state=42)

rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X_train, y_train)
preds = rfr.predict(X_test)

mse = mean_squared_error(y_test, preds)
mae = mean_absolute_error(y_test, preds)

print(f"text-embedding-3-small performance on prediction: mse={mse:.2f}, mae={mae:.2f}")

text-embedding-3-small performance on prediction: mse=0.02, mae=0.12


In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

features = np.hstack([
    np.vstack(df.embedding.values),
    np.vstack(df.embedding_b.values)
])

# Initialize model
rfr = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform k-fold cross validation
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
mse_scores = -cross_val_score(rfr, features, df.weight_a, 
                             scoring='neg_mean_squared_error', 
                             cv=kf)
mae_scores = -cross_val_score(rfr, features, df.weight_a, 
                             scoring='neg_mean_absolute_error', 
                             cv=kf)

# Print results
print(f"Cross-validation results ({n_folds} folds):")
print(f"MSE: {mse_scores.mean():.2f} (+/- {mse_scores.std() * 2:.2f})")
print(f"MAE: {mae_scores.mean():.2f} (+/- {mae_scores.std() * 2:.2f})")

In [18]:
df_test = pd.read_csv("data/test-part-1-processed.csv")

In [ ]:
test_features = np.hstack([
    np.vstack(df_test.embedding.values),
    np.vstack(df_test.embedding_b.values)
])
preds = rfr.predict(test_features)
df_test['predicted_weight'] = preds
df_test.to_csv("data/test-part-1-processed-predicted.csv", index=False)
